In [59]:
# loading libraries 
import pandas as pd
import numpy as np

In [60]:
#read and display data
df1 =pd.read_csv("resources/NY-House-Dataset.csv")
df2 = pd.read_csv('resources/housing_geocode_extraction.csv')
print(display(df1.head(5)))
print(display(df2.head(5)))

,BROKERTITLE,TYPE,PRICE,BEDS,BATH,PROPERTYSQFT,ADDRESS,STATE,MAIN_ADDRESS,ADMINISTRATIVE_AREA_LEVEL_2,LOCALITY,SUBLOCALITY,STREET_NAME,LONG_NAME,FORMATTED_ADDRESS,LATITUDE,LONGITUDE
0,Brokered by Douglas Elliman -111 Fifth Ave,Condo for sale,315000,2,2.000000,1400.0,2 E 55th St Unit 803,"New York, NY 10022","2 E 55th St Unit 803New York, NY 10022",New York County,New York,Manhattan,East 55th Street,Regis Residence,"Regis Residence, 2 E 55th St #803, New York, N...",40.761255,-73.974483
1,Brokered by Serhant,Condo for sale,195000000,7,10.000000,17545.0,Central Park Tower Penthouse-217 W 57th New Yo...,"New York, NY 10019",Central Park Tower Penthouse-217 W 57th New Yo...,United States,New York,New York County,New York,West 57th Street,"217 W 57th St, New York, NY 10019, USA",40.766393,-73.980991
2,Brokered by Sowae Corp,House for sale,260000,4,2.000000,2015.0,620 Sinclair Ave,"Staten Island, NY 10312","620 Sinclair AveStaten Island, NY 10312",United States,New York,Richmond County,Staten Island,Sinclair Avenue,"620 Sinclair Ave, Staten Island, NY 10312, USA",40.541805,-74.196109
3,Brokered by COMPASS,Condo for sale,69000,3,1.000000,445.0,2 E 55th St Unit 908W33,"Manhattan, NY 10022","2 E 55th St Unit 908W33Manhattan, NY 10022",United States,New York,New York County,New York,East 55th Street,"2 E 55th St, New York, NY 10022, USA",40.761398,-73.974613
4,Brokered by Sotheby's International Realty - E...,Townhouse for sale,55000000,7,2.373861,14175.0,5 E 64th St,"New York, NY 10065","5 E 64th StNew York, NY 10065",United States,New York,New York County,New York,East 64th Street,"5 E 64th St, New York, NY 10065, USA",40.767224,-73.969856


None


,LATITUDE,LONGITUDE,STATE,LOCALITY,SUBLOCALITY,POSTCODE,PROPERTY_CATEGORY
0,40.761255,-73.974483,New York,New York,Manhattan,10022,catering.restaurant
1,40.766393,-73.980991,New York,New York,Manhattan,10019,building.residential
2,40.541805,-74.196109,New York,New York,Staten Island,10312,building.residential
3,40.761398,-73.974613,New York,New York,Manhattan,10022,catering.bar
4,40.767224,-73.969856,New York,New York,Manhattan,10065,building


None


In [61]:
# Clean sublocality and state columns in both datasets
df1['SUBLOCALITY'] = df1['SUBLOCALITY'].str.strip().str.lower()
df1['STATE'] = df1['STATE'].str.strip().str.lower()
df2['SUBLOCALITY'] = df2['SUBLOCALITY'].str.strip().str.lower()
df2['STATE'] = df2['STATE'].str.strip().str.lower()



In [62]:
# Drop duplicates in the key column
df1 = df1.drop_duplicates(subset=['LATITUDE', 'LONGITUDE'])
df2 = df2.drop_duplicates(subset=['LATITUDE', 'LONGITUDE'])

In [63]:
# Merge datasets on LATITUDE and LONGITUDE
merged_df = pd.merge(df1, df2, on=['LATITUDE', 'LONGITUDE'], how='inner', suffixes=('_orig', '_ext'))

In [64]:
#Inspect Overlapping Columns

duplicate_cols = [col for col in merged_df.columns if col.endswith('_orig') or col.endswith('_ext')]
print(duplicate_cols)


['STATE_orig', 'LOCALITY_orig', 'SUBLOCALITY_orig', 'STATE_ext', 'LOCALITY_ext', 'SUBLOCALITY_ext']


In [65]:
merged_df = merged_df.drop(columns=['STATE_orig', 'LOCALITY_orig', 'SUBLOCALITY_orig'])

In [66]:
#check thecolumns names
merged_df.columns

Index(['BROKERTITLE', 'TYPE', 'PRICE', 'BEDS', 'BATH', 'PROPERTYSQFT',
       'ADDRESS', 'MAIN_ADDRESS', 'ADMINISTRATIVE_AREA_LEVEL_2', 'STREET_NAME',
       'LONG_NAME', 'FORMATTED_ADDRESS', 'LATITUDE', 'LONGITUDE', 'STATE_ext',
       'LOCALITY_ext', 'SUBLOCALITY_ext', 'POSTCODE', 'PROPERTY_CATEGORY'],
      dtype='object')

In [ ]:
# Select relevant columns
selected_columns = [
    'PRICE', 'BEDS', 'BATH', 'PROPERTYSQFT','STREET_NAME',
    'PROPERTY_CATEGORY','TYPE', 'SUBLOCALITY_ext','POSTCODE','STATE_ext','LATITUDE', 'LONGITUDE'
]
merged_df = merged_df[selected_columns]
merged_df.rename(columns={'STREET_NAME': 'borough'}, inplace=True)


In [68]:
#check the data shape
merged_df.shape

(4200, 12)

In [69]:
#check the merged data
merged_df.head()


,PRICE,BEDS,BATH,PROPERTYSQFT,STREET_NAME,PROPERTY_CATEGORY,TYPE,SUBLOCALITY_ext,POSTCODE,STATE_ext,LATITUDE,LONGITUDE
0,315000,2,2.000000,1400.0,East 55th Street,catering.restaurant,Condo for sale,manhattan,10022,new york,40.761255,-73.974483
1,195000000,7,10.000000,17545.0,New York,building.residential,Condo for sale,manhattan,10019,new york,40.766393,-73.980991
2,260000,4,2.000000,2015.0,Staten Island,building.residential,House for sale,staten island,10312,new york,40.541805,-74.196109
3,69000,3,1.000000,445.0,New York,catering.bar,Condo for sale,manhattan,10022,new york,40.761398,-73.974613
4,55000000,7,2.373861,14175.0,New York,building,Townhouse for sale,manhattan,10065,new york,40.767224,-73.969856


In [70]:
# Calculate price per square foot
merged_df['PRICE_PER_SQFT'] = merged_df['PRICE'] / merged_df['PROPERTYSQFT']

# Standardize type column
merged_df['TYPE'] = merged_df['TYPE'].str.lower().str.replace(' for sale', '', regex=False)

# Display cleaned and enhanced DataFrame
merged_df.head()

,PRICE,BEDS,BATH,PROPERTYSQFT,STREET_NAME,PROPERTY_CATEGORY,TYPE,SUBLOCALITY_ext,POSTCODE,STATE_ext,LATITUDE,LONGITUDE,PRICE_PER_SQFT
0,315000,2,2.000000,1400.0,East 55th Street,catering.restaurant,condo,manhattan,10022,new york,40.761255,-73.974483,225.000000
1,195000000,7,10.000000,17545.0,New York,building.residential,condo,manhattan,10019,new york,40.766393,-73.980991,11114.277572
2,260000,4,2.000000,2015.0,Staten Island,building.residential,house,staten island,10312,new york,40.541805,-74.196109,129.032258
3,69000,3,1.000000,445.0,New York,catering.bar,condo,manhattan,10022,new york,40.761398,-73.974613,155.056180
4,55000000,7,2.373861,14175.0,New York,building,townhouse,manhattan,10065,new york,40.767224,-73.969856,3880.070547


In [71]:
# # Standardize column names
# merged_df.columns = merged_df.columns.str.lower().str.replace(' ', '_')
# #check changes
# merged_df.columns

Check for Missing Values:

In [72]:
# Check for missing values in the selected columns
missing_data = merged_df.isnull().sum()
print("Missing Data by Column:")
missing_data


Missing Data by Column:


PRICE                  0
BEDS                   0
BATH                   0
PROPERTYSQFT           0
STREET_NAME            0
PROPERTY_CATEGORY    641
TYPE                   0
SUBLOCALITY_ext        0
POSTCODE               0
STATE_ext              0
LATITUDE               0
LONGITUDE              0
PRICE_PER_SQFT         0
dtype: int64

In [73]:
# Save the merged DataFrame to a CSV file
df_copy = merged_df.copy()

# Convert column names to lowercase in the original DataFrame
df_copy.columns = df_copy.columns.str.lower()
df_copy.to_csv('resources/Cleaned_data.csv', index=False)

# Confirm the file has been saved
print("File saved as 'Cleaned_data'")

File saved as 'Cleaned_data'


Handle Missing Values:

In [74]:
merged_df.columns

Index(['PRICE', 'BEDS', 'BATH', 'PROPERTYSQFT', 'STREET_NAME',
       'PROPERTY_CATEGORY', 'TYPE', 'SUBLOCALITY_ext', 'POSTCODE', 'STATE_ext',
       'LATITUDE', 'LONGITUDE', 'PRICE_PER_SQFT'],
      dtype='object')

In [75]:
#chech data type
merged_df.dtypes

PRICE                  int64
BEDS                   int64
BATH                 float64
PROPERTYSQFT         float64
STREET_NAME           object
PROPERTY_CATEGORY     object
TYPE                  object
SUBLOCALITY_ext       object
POSTCODE              object
STATE_ext             object
LATITUDE             float64
LONGITUDE            float64
PRICE_PER_SQFT       float64
dtype: object

In [76]:
# Check for whitespac
merged_df.columns = merged_df.columns.str.strip()

In [77]:
#drope the missing values
merged_df = merged_df.dropna(how='any')

In [78]:
# Check for duplicate rows
duplicates = merged_df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

# Remove duplicates
merged_df = merged_df.drop_duplicates()


Number of duplicate rows: 0


In [79]:
# Define a function to remove outliers using the IQR method
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)  # First quartile
    Q3 = df[column].quantile(0.75)  # Third quartile
    IQR = Q3 - Q1  # Interquartile range
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # Return the dataframe after filtering out outliers
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# List of columns to clean
columns_to_clean = ['PRICE', 'BEDS', 'BATH', 'PROPERTYSQFT', 'PRICE_PER_SQFT']

# Clean outliers for the relevant numerical columns using a loop
for col in columns_to_clean:
    merged_df = remove_outliers(merged_df, col)

# Display the cleaned dataset
merged_df.head()


,PRICE,BEDS,BATH,PROPERTYSQFT,STREET_NAME,PROPERTY_CATEGORY,TYPE,SUBLOCALITY_ext,POSTCODE,STATE_ext,LATITUDE,LONGITUDE,PRICE_PER_SQFT
0,315000,2,2.0,1400.000000,East 55th Street,catering.restaurant,condo,manhattan,10022,new york,40.761255,-73.974483,225.000000
2,260000,4,2.0,2015.000000,Staten Island,building.residential,house,staten island,10312,new york,40.541805,-74.196109,129.032258
3,69000,3,1.0,445.000000,New York,catering.bar,condo,manhattan,10022,new york,40.761398,-73.974613,155.056180
6,899500,2,2.0,2184.207862,Manhattan,building,condo,manhattan,10027,new york,40.809448,-73.946777,411.819779
8,265000,1,1.0,750.000000,Morrison Avenue,building.residential,co-op,the bronx,10473,new york,40.821586,-73.874089,353.333333


In [80]:
# check data type
merged_df.dtypes

PRICE                  int64
BEDS                   int64
BATH                 float64
PROPERTYSQFT         float64
STREET_NAME           object
PROPERTY_CATEGORY     object
TYPE                  object
SUBLOCALITY_ext       object
POSTCODE              object
STATE_ext             object
LATITUDE             float64
LONGITUDE            float64
PRICE_PER_SQFT       float64
dtype: object

In [81]:
#If any column names have leading or trailing spaces
merged_df.columns = merged_df.columns.str.strip()
# Correct Inconsistent Value
merged_df['STATE_ext'] = merged_df['STATE_ext'].str.title()
merged_df['SUBLOCALITY_ext'] = merged_df['SUBLOCALITY_ext'].str.title()


In [82]:
merged_df.head()

,PRICE,BEDS,BATH,PROPERTYSQFT,STREET_NAME,PROPERTY_CATEGORY,TYPE,SUBLOCALITY_ext,POSTCODE,STATE_ext,LATITUDE,LONGITUDE,PRICE_PER_SQFT
0,315000,2,2.0,1400.000000,East 55th Street,catering.restaurant,condo,Manhattan,10022,New York,40.761255,-73.974483,225.000000
2,260000,4,2.0,2015.000000,Staten Island,building.residential,house,Staten Island,10312,New York,40.541805,-74.196109,129.032258
3,69000,3,1.0,445.000000,New York,catering.bar,condo,Manhattan,10022,New York,40.761398,-73.974613,155.056180
6,899500,2,2.0,2184.207862,Manhattan,building,condo,Manhattan,10027,New York,40.809448,-73.946777,411.819779
8,265000,1,1.0,750.000000,Morrison Avenue,building.residential,co-op,The Bronx,10473,New York,40.821586,-73.874089,353.333333


In [83]:
merged_df['PROPERTY_CATEGORY'].unique()

array(['catering.restaurant', 'building.residential', 'catering.bar',
       'building', 'building.garage', 'commercial;rental.storage',
       'catering.fast_food', 'accommodation.hotel',
       'public_transport.bus', 'tourism.sights', 'sport.sports_centre',
       'building.commercial', 'commercial',
       'commercial.food_and_drink.bakery', 'catering.ice_cream',
       'commercial;service.beauty.hairdresser',
       'commercial.furniture_and_interior', 'commercial.pet;pet.shop',
       'commercial.health_and_beauty;service.beauty.spa',
       'healthcare.hospital', 'education.university', 'catering.cafe',
       'commercial.convenience', 'building.office', 'building.healthcare',
       'leisure.playground', 'commercial.clothing',
       'commercial.discount_store', 'service.vehicle.fuel',
       'commercial;service.cleaning.dry_cleaning'], dtype=object)

In [84]:
# Define mapping to major categories
# Define mapping to major categories
category_mapping = {
    'building.residential': 'Residential',
    'accommodation.hotel': 'Hospitality',
    'healthcare.hospital': 'Other',  # 
    'building.commercial': 'Commercial',
    'commercial': 'Commercial',
    'commercial;service.beauty.hairdresser': 'Commercial',
    'commercial.health_and_beauty;service.beauty.spa': 'Commercial',
    'commercial.furniture_and_interior': 'Commercial',
    'commercial.pet;pet.shop': 'Commercial',
    'commercial.convenience': 'Commercial',
    'catering.restaurant': 'Commercial',
    'catering.bar': 'Commercial',
    'catering.fast_food': 'Commercial',
    'catering.ice_cream': 'Commercial',
    'catering.cafe': 'Commercial',
    'building.office': 'Commercial',
    'building.healthcare': 'Other',  
    'commercial.clothing': 'Commercial',
    'commercial.discount_store': 'Commercial',
    'service.vehicle.fuel': 'Commercial',
    'commercial;rental.storage': 'Commercial',
    'public_transport.bus': 'Other',
    'tourism.sights': 'Other',
    'sport.sports_centre': 'Other',
    'leisure.playground': 'Other',
    'education.university': 'Other',
    'building.garage': 'Other',
    'building': 'Residential',  
    'commercial;service.cleaning.dry_cleaning': 'Commercial',
}
# Function to map categories
def map_category(category):
    return category_mapping.get(category, 'Other')

# Apply the mapping
merged_df['CATEGORY'] = merged_df['PROPERTY_CATEGORY'].apply(map_category)

# Display the updated DataFrame
merged_df[['PROPERTY_CATEGORY', 'CATEGORY']]

,PROPERTY_CATEGORY,CATEGORY
0,catering.restaurant,Commercial
2,building.residential,Residential
3,catering.bar,Commercial
6,building,Residential
8,building.residential,Residential
...,...,...
4192,building,Residential
4195,building,Residential
4197,building,Residential
4198,building,Residential


In [85]:
merged_df['CATEGORY'].unique()

array(['Commercial', 'Residential', 'Other', 'Hospitality'], dtype=object)

In [86]:
# #Drop any rows with missing or unclassified values:
merged_df = merged_df.dropna(subset=['TYPE', 'CATEGORY'])


In [87]:
# #drop propertycategory 
del merged_df['PROPERTY_CATEGORY']

In [88]:
merged_df.shape

(2716, 13)

In [89]:
merged_df.head()

,PRICE,BEDS,BATH,PROPERTYSQFT,STREET_NAME,TYPE,SUBLOCALITY_ext,POSTCODE,STATE_ext,LATITUDE,LONGITUDE,PRICE_PER_SQFT,CATEGORY
0,315000,2,2.0,1400.000000,East 55th Street,condo,Manhattan,10022,New York,40.761255,-73.974483,225.000000,Commercial
2,260000,4,2.0,2015.000000,Staten Island,house,Staten Island,10312,New York,40.541805,-74.196109,129.032258,Residential
3,69000,3,1.0,445.000000,New York,condo,Manhattan,10022,New York,40.761398,-73.974613,155.056180,Commercial
6,899500,2,2.0,2184.207862,Manhattan,condo,Manhattan,10027,New York,40.809448,-73.946777,411.819779,Residential
8,265000,1,1.0,750.000000,Morrison Avenue,co-op,The Bronx,10473,New York,40.821586,-73.874089,353.333333,Residential


In [90]:
df_copy = merged_df.copy()

# Convert column names to lowercase in the original DataFrame
df_copy.columns = df_copy.columns.str.lower()

# Save the merged DataFrame to a CSV file
df_copy.to_csv('resources/Cleaned_data_without_outliers.csv', index=False)

# Confirm the file has been saved
print("File saved as 'Cleaned_data_without_outliers'")


File saved as 'Cleaned_data_without_outliers'
